# Geometry Optimization

We present here a complete example on how to use QMCTorch to optimize the geometry of a H2 molecule.
As previously the first task is to import all the modules needed

In [ ]:
from torch import optim
from torch.optim import Adam
from qmctorch.wavefunction import SlaterJastrow
from qmctorch.solver import Solver
from qmctorch.sampler import Metropolis
from qmctorch.scf import Molecule
from qmctorch.utils import plot_energy
from qmctorch.utils import set_torch_double_precision
set_torch_double_precision()

We then define the molecule. We create a H2 molecule with a short atomic distance that we will then try to relax


In [ ]:
mol = Molecule(atom = 'H 0. 0. -0.5; H 0. 0. 0.5', unit='bohr', 
               calculator='pyscf', basis='sto-3g', redo_scf=True)

As for the wave function optimization example we need to define a wave function, a sampler an optimizer and a solver.

In [ ]:
# wave function with only the ground state determinant
wf = SlaterJastrow(mol, configs='single_double(2,2)').gto2sto()

# sampler
sampler = Metropolis(nwalkers=1000, nstep=200, step_size=0.5,
                     nelec=wf.nelec, ndim=wf.ndim,
                     ntherm=-1, ndecor=100,
                     init=mol.domain('normal'),
                     move={'type': 'all-elec', 'proba': 'normal'})
# optimizer
opt = Adam(wf.parameters(), lr=0.005)

# scheduler
scheduler = optim.lr_scheduler.StepLR(opt, step_size=20, gamma=0.75)

In [ ]:
solver = Solver(wf=wf,
                       sampler=sampler,
                       optimizer=opt,
                       scheduler=None)
solver.configure(loss='energy', grad='auto', track=['local_energy','geometry'], freeze = ['ao'])

Note that we must set `grad='auto'` to perform a geometry optimization. To optimize the geometry of the molecule we must use the `set_params_requires_grad` method of the solver. We here set all the wave function parameters and all the atomic positions as variational parameters, leading to a simultaneous optimization of the wave function and of the atomic coordinates. We can then run the optimization here using 50 epochs

In [ ]:
solver.set_params_requires_grad(wf_params=False, geo_params=True)
obs = solver.run(50)


In [ ]:
solver.save_traj('h2_traj.xyz', obs)

In [ ]:
plot_energy(obs.local_energy)